In [54]:
import pyomo.environ as pyo
import pandas as pd
from pathlib import Path
import preprocess_data as dat

data_path = Path("../data")
year = 2021

## Data

In [55]:
adjacent_list = dat.create_adjacent_list(data_path)
adjacent_matrix = dat.gen_adjacent_matrix(adjacent_list)
df_init, counties = dat.data_ingest(data_path, year)
df = dat.create_df(df_init, counties, adjacent_list)
model = pyo.AbstractModel()

## Defs

In [56]:
def param_adjacent(m, i, j):
    return int(j in adjacent_matrix[i])

def param_pop(m, i):
    return df["population"][i - 1]

def con_a(m, i):
    return sum((m.a[i, j] * m.x[j]) for j in m.J) >= m.y[i]

def con_x(m):
    return sum(m.x[j] for j in m.J) <= m.k

def obj_sum(m):
    return pyo.summation(m.p, m.y)

## Constraints

In [57]:
# value of n (number of counties)
model.n = 88

# TOSET limit on number of pirincipal places of buisnesses opened (init to 5)
model.k = 5

# range of i and j (iterating over counties)
model.I = pyo.RangeSet(1, model.n)
model.J = pyo.RangeSet(1, model.n)

model.p = pyo.Param(model.I, initialize=param_pop)  # population of county i

# model.a = pyo.Set(model.I, model.J, within=pyo.Binary, initialize=param_adjacent)  # 1 if county i and j are adjacent
model.a = pyo.Var(model.I, model.J, domain=pyo.Binary, initialize=param_adjacent)  # 1 if county i and j are adjacent

model.x = pyo.Var(model.J, domain=pyo.Binary)  # 1 if principal place of business is opened in county j
model.y = pyo.Var(model.I, domain=pyo.Binary)  # 1 if county i is covered

model.obj = pyo.Objective(rule=obj_sum, sense=pyo.maximize)

# model.a_constraint = pyo.Constraint(model.I, rule=con_a)
model.x_constraint = pyo.Constraint(rule=con_x)

## Solve

In [58]:
instance = model.create_instance()
opt = pyo.SolverFactory("cbc")
results = opt.solve(instance).write()
results
# instance.pprint()
# instance.solutions.store_to(results)
# model21.pprint()
# model21.x.get_values()
# solution.print_data()
# solution.solutions.store_to(results)

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 11769923.0
  Upper bound: 11769923.0
  Number of objectives: 1
  Number of constraints: 0
  Number of variables: 0
  Number of binary variables: 176
  Number of integer variables: 176
  Number of nonzeros: 0
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch an

In [1]:
instance.x.pprint()

NameError: name 'instance' is not defined